<font size="6">Улучшение сходимости нейросетей и борьба с переобучением</font>

# Сигмоида затухает и теоретически и практически

Посмотрим на практике, загрузим данные, создадим сеть, обучим ее и посмотрим, как проходит обучение. 

Загрузим датасет MNIST:

In [ ]:
from torchvision.datasets import MNIST
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from IPython.display import clear_output

# transforms for data
transform = torchvision.transforms.Compose(
    [torchvision.transforms.ToTensor(),
     torchvision.transforms.Normalize((0.5), (0.5))])

train_set = MNIST(root='./MNIST', train=True, download=True, transform=transform)
test_set = MNIST(root='./MNIST', train=False, download=True, transform=transform)

batch_size = 32
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=2)

clear_output()
print("Already downloaded!")

Создадим сеть с сигмоидой в качестве функции активации:

In [ ]:
import torch.nn as nn

class SimpleMNIST_NN(nn.Module):
    def __init__(self, n_layers, activation=nn.Sigmoid):
        super().__init__()
        self.n_layers = n_layers # Num of layers
        self.activation = activation()
        layers = [nn.Linear(28 * 28, 100), self.activation] # input layer
        for _ in range(n_layers - 1): # append num of layers
            layers.append(nn.Linear(100, 100))
            layers.append(self.activation)
        layers.append(nn.Linear(100, 10)) # 10 classes
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        x = x.view(-1, 28 * 28) # reshape to [-1, 784]
        x = self.layers(x)
        return x

Код для визуализации результатов обучения моделей:

In [ ]:
def exponential_smoothing(scalars, weight):  
    last = scalars[0]  
    smoothed = []
    for point in scalars:
        smoothed_val = last * weight + (1 - weight) * point  
        smoothed.append(smoothed_val)                        
        last = smoothed_val                                 

    return smoothed

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_history(history, n_epochs=5, smooth_val=0.90):
    fig, ax =  plt.subplots(3, 1, figsize=(12, 14))
    for stage_idx, (stage_lbl, stage_title) in enumerate(
        zip(['loss_on_train', 'loss_on_test'],
            ['train loss', 'test loss'])):
        # plot history on each learning step
        epoch_len = len(history[stage_lbl])//n_epochs
        full_stage_len = len(history[stage_lbl])
        ax[stage_idx].plot(exponential_smoothing(history[stage_lbl], smooth_val),
                           label='smoothed',
                           color='m')
        ax[stage_idx].plot(history[stage_lbl],
                           label='raw',
                           alpha=0.2,
                           color='c')
        ax[stage_idx].set_title(stage_title)
        ax[stage_idx].set_xlabel('epochs')
        ax[stage_idx].set_ylabel('loss')
        epochs_ticks_positions = np.arange(stop=full_stage_len+1,
                                           step=epoch_len)
        ax[stage_idx].set_xticks(epochs_ticks_positions)
        ax[stage_idx].set_xticklabels(np.arange(n_epochs+1))
        ax[stage_idx].legend()

        # plot mean train and test loss combined
        mean_loss_on_epoch = [np.mean(history[stage_lbl][i:i+epoch_len]) \
                              for i in range(0, full_stage_len, epoch_len)]
        std_loss_on_epoch = [np.std(history[stage_lbl][i:i+epoch_len]) \
                              for i in range(0, full_stage_len, epoch_len)]

        ax[2].set_title('\nAverage loss per epoch')
        ax[2].errorbar(np.arange(n_epochs) + stage_idx / 30.,
                       mean_loss_on_epoch,
                       yerr=std_loss_on_epoch,
                       capsize=5,
                       fmt="X--",
                       label=stage_title)
        ax[2].set_xticks(np.arange(5))
        ax[2].set_xticklabels(np.arange(5))
        ax[2].set_xlabel('epochs')
        ax[2].set_ylabel('loss')
        ax[2].legend()

    fig.suptitle(history['model_name'], fontsize=24)
    plt.show()

Функция для обучения сети на обучающей выборке:

In [ ]:
import numpy as np
import torch

# compute on cpu or gpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def train_epoch(model,
                optimizer,
                criterion,
                train_loader):
    loss_history = []
    for batch in train_loader: 
        optimizer.zero_grad()
        x_train, y_train = batch # parse data
        x_train, y_train = x_train.to(device), y_train.to(device) # compute on gpu
        y_pred = model(x_train) # get predictions
        loss = criterion(y_pred, y_train) # compute loss
        loss_history.append(loss.cpu().detach().numpy()) # write loss to log
        loss.backward()
        optimizer.step()
    return loss_history

Функция для валидации сети на валидационной выборке:

In [ ]:
def validate(model,
             criterion,
             val_loader):
    cumloss = 0
    loss_history = []
    with torch.no_grad():
        for batch in val_loader:
            x_train, y_train = batch # parse data
            x_train, y_train = x_train.to(device), y_train.to(device) # compute on gpu
            y_pred = model(x_train) # get predictions
            loss = criterion(y_pred, y_train) # compute loss
            loss_history.append(loss.cpu().detach().numpy()) # write loss to log
            cumloss += loss
    return cumloss / len(val_loader), loss_history # mean loss and history

Функция для обучения модели:

In [ ]:
from tqdm import tqdm

def train_model(model, optimizer, model_name=None, n_epochs=5):
  
    criterion = nn.CrossEntropyLoss().to(device)

    train_history = {}
    train_history['model_name'] = model_name
    train_history['loss_on_train'] = []
    train_history['loss_on_test'] = []

    for epoch in tqdm(range(n_epochs)):
        loss_on_train = train_epoch(model,
                                    optimizer,
                                    criterion,
                                    train_loader)
        _, loss_on_test = validate(model,
                                   criterion,
                                   test_loader)
        train_history['loss_on_train'].extend(loss_on_train)
        train_history['loss_on_test'].extend(loss_on_test)
    return train_history

Создадим и запустим обучение модели с двумя слоями:

In [ ]:
import torch.optim as optim

model = SimpleMNIST_NN(n_layers=2).to(device) 
optimizer = optim.SGD(model.parameters(), lr=0.001)

history = train_model(model, optimizer, model_name='n_layers2_sigmoid')
plot_history(history)

In [ ]:
model = SimpleMNIST_NN(n_layers=3).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.001)
history = train_model(model, optimizer, model_name='n_layers3_sigmoid')
plot_history(history)

Нейросеть с тремя слоями вообще не учится. Почему? Можем попробовать разобраться.

Для этого напишем функции, которые будут следить за распределением градиентов и активаций на наших слоях.

И скажем при помощи метода register_backward_hook пайторчу исполнять эти функции при каждом пропускании градиента.

In [ ]:
from collections import defaultdict

def get_forward_hook(history_dict, key):

    def forward_hook(self, input_, output):
        history_dict[key].append(input_[0].abs().mean().cpu().detach().numpy())
    return forward_hook


def get_backward_hook(history_dict, key):

    def backward_hook(grad):  # for tensors
        history_dict[key].append(grad.abs().mean().cpu().detach().numpy())
    return backward_hook

def register_model_hooks(model, max_ind=4):
    cur_ind = 0
    hooks_data_history = defaultdict(list)
    for child in model.layers.children():
        if isinstance(child, nn.Linear):
            cur_ind += 1
            forward_hook = get_forward_hook(hooks_data_history,
                                            f"activation_{cur_ind}")
            child.register_forward_hook(forward_hook)

            backward_hook = get_backward_hook(hooks_data_history,
                                              f"gradient_{max_ind - cur_ind + 1}")
            child.weight.register_hook(backward_hook)
    return hooks_data_history

Запустим обучение модели с 3 слоями:

In [ ]:
model = SimpleMNIST_NN(n_layers=3).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.001)

hooks_data_history = register_model_hooks(model)

history = train_model(model, optimizer, model_name="n_layers3_sigmoid2")

In [ ]:
def plot_hooks_data(hooks_data_history, smooth_val=0.85):
    keys = hooks_data_history.keys()
    n_layers = len(keys) // 2
    fig, ax = plt.subplots(2, n_layers, figsize=(12, 12), sharey='row')
    for layer_idx in range(n_layers):
        
        smoothed_values = exponential_smoothing(hooks_data_history[f'activation_{layer_idx + 1}'],
                                                smooth_val) 
        ax[0, layer_idx].plot(smoothed_values)
        ax[0, layer_idx].set_title(f'activation_{layer_idx + 1}')
        ax[0, layer_idx].set_ylim([0, 1])

        smoothed_values = exponential_smoothing(hooks_data_history[f'gradient_{layer_idx + 1}'],
                                                smooth_val) 
        scale_transform = lambda x : np.array(x) * 1.e5 + 1
        ax[1, layer_idx].plot(scale_transform(smoothed_values))
        ax[1, layer_idx].set_title(f'gradient_{layer_idx + 1}')
        ax[1, layer_idx].set_yscale('log')
    ax[0,0].set_ylabel('value')
    ax[1,0].set_ylabel(r'value $\times$ $10^5 + 1$')
    fig.tight_layout()
    plt.show()

In [ ]:
plot_hooks_data(hooks_data_history)

Мы видим, что градиент нашей модели стремительно затухает. Первые слои (до которых градиент доходит последним), получают значения градиента, мало отличимые от нуля.

Причем, это будет верно с самых первых шагов обучения нашей модели



Это явление получило название **паралич сети**

# Затухание градиента

Откуда оно берется? 

Посмотрим на обычную сигмоиду

$$\sigma(z) = \dfrac 1 {1 + e^{-z}}$$

Ее производная, как вы уже выводили, равна

$$\dfrac {\delta \sigma(z)} {\delta z} = \sigma(z) (1 - \sigma(z))$$

Какое максимальное значение у такой функции?

Учтем, что сигмоида находится в пределах от 0 до 1

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.style.use('seaborn-whitegrid')

x = np.arange(0, 1.001, 0.001)
plt.plot(x, x - x**2)
plt.title('Derivative of the sigmoidal function', size=15)
plt.show()

Получается, что максимальное значение производной по сигмоиде - 1/4

Теперь возьмем простую нейронную сеть 

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/out/simple_nn_with_sigmoid.png" width="750">

Посчитаем у нее градиент

$$\dfrac {\delta L} {\delta z_4} = \dfrac {\delta L} {\delta y} \dfrac {\delta y} {\delta z_4} = \dfrac {\delta L} {\delta y} \dfrac {\delta \sigma(w_5z)} {\delta z} w_5 \le \dfrac 1 4 \dfrac {\delta L} {\delta y}  w_5 $$

Аналогично можно посчитать градиент для $z_3$

$$\dfrac {\delta L} {\delta z_3} = \dfrac {\delta L} {\delta z_4} \dfrac {\delta z_4} {\delta z_3} \le \dfrac {\delta L} {\delta y} \dfrac {\delta \sigma(w_4z)} {\delta z} w_5 \le \left({\dfrac 1 4}\right)^2 \dfrac {\delta L} {\delta y}  w_5 w_4$$

И так далее

$$\dfrac {\delta L} {\delta x}  \le \left({\dfrac 1 4}\right)^5 \dfrac {\delta L} {\delta y}  w_5 w_4 w_3 w_2 w_1$$

Таким образом, градиент начинает экспоненциально затухать, если веса маленькие.

Если веса большие - то градиент наоборот начнет экспоненциально возрастать - все взорвется

Для некоторых функций активации картина будет не столь катастрофична, но тоже неприятна.

При выполнении заданий вы посмотрите, например, как ведет себя функция ReLU в этом случае.

# Инициализация весов

Одним из способов борьбы с затухающим градиентом является правильная инициализация весов. Как это сделать?

**Идея 1:** инициализировать все веса константой. 

Проблема: градиент во всем весам будет одинаков, как и обновление весов. Все нейроны в слое будут учить одно и то же, или, в случае $const = 0$, [не будут учиться вообще](https://habr.com/ru/post/592711/).


Вывод: в качестве начальных весов нужно выбирать различные значения.



**Идея 2:** инициализировать веса нормальным (Гауссовским) шумом с матожиданием 0 и маленькой дисперсией. 

Маленькая дисперсия нужна, чтобы не получить огромные градиенты за большие изначальные ошибки в предсказании.

In [ ]:
# Normal distribution: mu = 0, sigma = 1

x = np.arange(-4, 4.1, 0.1)
y = np.exp(-np.square(x)/2) / np.sqrt(2*np.pi)

plt.style.use('seaborn-whitegrid')
plt.title('Normal distribution: mu = 0, sigma = 1', size=15)
plt.plot(x, y)
plt.show()

Проблема: инициализация нормальным шумом не гарантирует отсутствие взрыва или затухания градиета.

**Идея 3:** формализуем условия при которых не будет происходить взрыв градиентов.

Нам бы хотелось, чтобы 
дисперсии признаков, получаемых на каждом слое были бы одинаковы: 

$$Dz^i = Dz^j. \tag{1}$$

И чтобы  начальные дисперсии градиентов для разных слоев были бы одинаковы:
 
$$D\dfrac {\delta L} {\delta z^i} = D\dfrac {\delta L} {\delta z^j}. \tag{2}$$

При выполнении этих условий градиент не затухает и не взрывается.

Попытаемся выполнить эти условия.

## Инициализация Ксавьера (Xavier Glorot)

### Вывод Xavier

Рассмотрим функцию активации гиперболический тангенс (tanh).
<img src ="https://edunet.kea.su/repo/EduNet-content/L07/out/simple_nn_with_tanh.png" width="600">

Это - [нечетная функция](https://ru.wikipedia.org/wiki/%D0%A7%D1%91%D1%82%D0%BD%D0%BE%D1%81%D1%82%D1%8C_%D1%84%D1%83%D0%BD%D0%BA%D1%86%D0%B8%D0%B8) с единичной производной в нуле. Функция и ее производная изображены ниже.

In [ ]:
x = np.arange(-10, 10.1, 0.1)
y = np.tanh(x)
dy = 1 / np.cosh(x)

plt.style.use('seaborn-whitegrid')
fig, (im1, im2) = plt.subplots(2, 1, figsize=(7, 7))
im1.set(title = "tanh(x)")
im1.plot(x[0:51], y[0:51], 'r', x[50:96], y[50:96], 'b', 
         x[95:106], y[95:106], 'g', x[105:151], y[105:151], 'b', 
         x[150:201], y[150:201], 'r') 
im2.set(title = "tanh'(x)")
im2.plot(x[0:51], dy[0:51], 'r', x[50:96], dy[50:96], 'b',
         x[95:106], dy[95:106], 'g', x[105:151], dy[105:151], 'b',
         x[150:201], dy[150:201], 'r')
plt.show()

При выборе весов нам важно не попасть в красные зоны с почти нулевой производной, т.к. в этих областях градиент затухает. Мы хотим инициализировать веса таким образом, чтобы признаки, поступающие на слой активации, находились в зеленой области в окрестности нуля. Матожидание признаков, поступающих на слой активации, будет равно нулю
$$E(z^i_t w_{kt})=0 \tag{3}.$$



Выход слоя активации на текущем слое будет зависеть от выхода слоя активации на предыдущем: $$z^{i+1} = f(z^iW^i). \tag{4}$$

В окрестности нуля функцию гиперболический тангенс можно считать [линейной](https://ru.wikipedia.org/wiki/%D0%A0%D1%8F%D0%B4_%D0%A2%D0%B5%D0%B9%D0%BB%D0%BE%D1%80%D0%B0):
$$z^{i+1} \approx z^i W^i \tag{5}.$$
Матожидаение выхода функции активации также равно нулю:
$$E(z^i_t)=0 \tag{6}$$

Значения весов в слоях будут генерироваться генератором случайных чисел, поэтому веса в различных слоях можно считать независимыми случайными величинами. Для независимых случайных величин выполняются формулы: 
1. Дисперсии суммы двух независимых величин:

$$D(\eta + \gamma) = D\eta + D\gamma \tag{7}.$$

2. Дисперсии произведения двух независимых величин:

$$D\eta\gamma = E(\eta\gamma)^2 - (E\eta\gamma)^2 = E\eta^2E\gamma^2 - (E\eta)^2(E\gamma)^2 \tag{8}.$$ 


Распишем условие $(1)$ с использованием $(5)$ и $(7)$:

$$D(z^{i+1}_{k}) = D(\sum_t z^i_t w_{kt}) = \sum_t D(z^i_t w_{kt}) \tag{9}$$

Дисперсии признаков на входе функции активации берем одинаковые:

$$D(z^{i+1}_{k}) = n D(z^i_0 w_{k0}),\tag{10}$$

где $n$ - размерность выхода слоя.

Применяем нашу формулу $(8)$ и получаем:

$$D(z^{i+1}_{k}) = n [E(z^i_0)^2E(w_{k0})^2 - (Ez^i_0)^2(Ew_{k0})^2]. \tag{11}$$

Используем $(6)$:

$$D(z^{i+1}_{k}) =   n E(z^i_0)^2E(w_{k0})^2. \tag{12}$$

Матожидание выходов активаций и весов равны 0. Из этого: 

$$D(z^{i}_{0}) = E(z^{i}_{0})^2 - (Ez^{i}_{0})^2 = E(z^{i}_{0})^2, \tag{13}$$

$$D(w_{k0}) = E(w_{k0})^2 - (Ew_{k0})^2 = E(w_{k0})^2.$$


Подставляем в $(12)$:

$$D(z^{i+1}_{k}) = n D(z^i_0)D(w_{k0}). \tag{14}$$

Из $(14)$ следует формула для зависимости выхода активаций любого слоя от весов предыдущих слоев и дисперсии исходных данных:

$$Dz^i = Dx \prod_{p=0}^{i-1}n_pDW^p, \tag{15}$$

где $n_p$ - размерность выхода слоя p-го слоя.

Аналогично можно вывести формулу для градиентов по активациям

$$D(\dfrac {\delta L} {\delta z^i}) = D(\dfrac {\delta L} {\delta z^d} ) \prod_{p=i}^{d}n_{p+1}DW^p. \tag{16}$$

Вспоминаем условия $(1)$, $(2)$:

$$Dz^i = Dz^j \tag{1},$$

$$D\dfrac {\delta L} {\delta z^i} = D\dfrac {\delta L} {\delta z^j} \tag{2}.$$

С учетом $(15)$, $(16)$ они эквивалентны условиям:

$$n_iDW^i = 1 \tag{17}$$

$$n_{i+1}DW^i = 1 \tag{18}$$

Условия могут быть невыполнимы одновременно:

 $$n_i \ne n_{i+1}. $$

Возьмем компромисс - среднее гармоническое решений первого и второго уравнения:

$$DW^i = \dfrac 2 {n_i + n_{i+1}} \tag{19}.$$

**Итого:** нам нужно инициализировать веса нейронов случайными величинами со следующим матожиданием и дисперсией:

$$ EW^i = 0,$$

$$DW^i = \dfrac 2 {n_i + n_{i+1}}.$$

При инициализации Xavier используется [равномерное распределение](https://ru.wikipedia.org/wiki/%D0%9D%D0%B5%D0%BF%D1%80%D0%B5%D1%80%D1%8B%D0%B2%D0%BD%D0%BE%D0%B5_%D1%80%D0%B0%D0%B2%D0%BD%D0%BE%D0%BC%D0%B5%D1%80%D0%BD%D0%BE%D0%B5_%D1%80%D0%B0%D1%81%D0%BF%D1%80%D0%B5%D0%B4%D0%B5%D0%BB%D0%B5%D0%BD%D0%B8%D0%B5): 

$$W_i \sim U[a, b ],$$

где $a=-b$, так как матожидание равно 0. 

Дисперсия которого выражается формулой: 
$$D(U[a, b]) = \dfrac 1 {12} (b -a)^2 = \dfrac 4 {12} b^2 = \dfrac 1 {3} b^2.$$

Из $(19)$ получим:

$$ b = \sqrt{\dfrac {6} {n_i + n_{i + 1}}}$$

**Инициализация  Xavier** - это инициализация весов случайной величиной с распределением:

$$W_i \sim U[-\sqrt{\dfrac {6} {n_i + n_{i + 1}}}, \sqrt{\dfrac {6} {n_i + n_{i + 1}}}],$$ 

где $n_i$ - размерность выхода слоя n-го слоя.



Чтобы понять, что происходит с выходами слоя активации при использовании инициализации Xavier, рассмотрим картинку из оригинальной статьи [Xavier, Yoshua, "Understanding the difficulty of training deep feedforward neural networks", Aistats, 2010](http://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf):

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L07/xavier_procentile_and_deviation_with_and_without_init.png" width="600">

На картинке изображена зависимость 98-[процентиля](https://en.wikipedia.org/wiki/Percentile) (отдельные маркеры) и стандартного отклонения (соединенные маркеры) значений на выходе слоя активации $tanh$ от эпохи обучения для различных слоев нейросети. 

Верхнее изображение - инициализация весов с помощью нормального распределения $W_i \sim U[-\dfrac {1} {\sqrt{n_i}}, \dfrac {1} {\sqrt{n_i}} ]$, нижнее с использованием инициализации Xavier.

На верхнем изображении видно, как значения 98-процентиля уходят в значения +1 и -1 (сначала на выходе первого слоя, потом на выходе второго и т.д.). Это значит, что для части нейронов происходит затухание градиентов (они переходят в область, отмеченную на графиках $tanh(x)$, $tanh’(x)$ красным). На нижней картинке такого не происходит. 

## Хи-инициализация (Kaiming He)

Для функции активации  ReLU и ее модификаций (PReLU, Leaky ReLU и т.д.) аналогично инициализации Xavier можно расписать условия $(1)$, $(2)$. Так вводится He-инициализация. 

### Вывод Kaiming

Аналогично выводу для Xavier получаем выражения:

$$Dz^i = Dx \prod_{p=0}^{i-1}\dfrac 1 2 n_pDW^p $$

$$D(\dfrac {\delta L} {\delta z^i}) = D(\dfrac {\delta L} {\delta z^d} ) \prod_{p=i}^{d}\dfrac 1 2 n_{p+1}DW^p $$

где $n_p$ - размерность выхода слоя p-го слоя.

Условия $(1)$, $(2)$ эквивалентны условиям: 

$$  \dfrac {n_iDW^i} {2}  = 1, $$

$$\dfrac {n_{i+1}DW^i} {2} = 1.$$

Можно опять взять среднее гармоническое. Но на практике берут либо $ \frac 2 {n_i}$, либо $\frac 2 {n_i + 1}$

Итого получим:

$$W^i \sim N(0, sd=\sqrt{\frac 2 n_i})$$

Более подробно с выводом Хи-инициализации можно ознакомиться в оригинальной [статье](https://arxiv.org/pdf/1502.01852v1.pdf).

## Важность инициализации весов

1. Нейросеть может сойтись значительно быстрее


<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L07/weight_initialization_influence_convergence_neural_networks.png" width="550">

2. В зависимости от выбранной активации сеть вообще может сойтись или не сойтись

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L07/activation_function_influence_convergence_neural_networks.png" width="550">

## Обобщение инициализаций Ксавьера и Хи-инициализации

Вообще говоря, коэффициенты в инициализациях (числитель в формуле для дисперсии), зависят от конкретной выбранной функции активации.
[В pytorch есть функции для вычисления этих коэффициентов](https://pytorch.org/docs/stable/nn.init.html)


## Ортогональная инициализация

Также иногда используется так называемая ортогональная инициализация весов. 
Для каждого слоя мы убеждаемся, что изначальная матрица весов является ортогональной. 

Выберем ортогональную матрицу весов 
$$W: WW^T = 1$$

Тогда:
1.  норма активации сохраняется (опять же, активации между слоями остаются в одном масштабе)
$$||s_{i+1}|| = ||W_{i}s_i|| = ||s_i||$$

2.  все нейроны делают «разные» преобразования
$$ ⟨W_i, W_j⟩ = 0~i \ne j$$
$$ ⟨W_i, W_j⟩ = 1~i = j$$


Иногда такая инициализация обеспечивает значительно лучшую сходимость, [тут можно почитать об этом подробнее](https://datascience.stackexchange.com/questions/64899/why-is-orthogonal-weights-initialization-so-important-for-ppo) 

## Инициализация весов в Pytorch

Для инициализации весов Pytorch используется модуль torch.nn.init

В нем определены разные функции для инициализации весов.

Нюанс состоит в том, что обычно для слоев разного типа может требоваться разная инициализация. Потому в функции, которая инициализирует слои вашей нейронной сети желательно прописывать разное поведение для разных слоев.

Попробуем, например, добавить в нашу нейросеть инициализацию. Нам нужна регуляризация Xavier, так как у нас Sigmoid

`torch.nn.init.calculate_gain` - Возвращает рекомендуемое значение коэффициента масштабирования для стандартного отклонения заданной функции активации.

In [ ]:
class SimpleMNIST_NN(nn.Module):
    def __init__(self,
                 n_layers,
                 activation=nn.Sigmoid,
                 init_form="normal"):
        super().__init__()
        self.n_layers = n_layers
        self.activation = activation()
        layers = [nn.Linear(28 * 28, 100), self.activation]
        for _ in range(0, n_layers - 1):
            layers.append(nn.Linear(100, 100))
            layers.append(self.activation)
        layers.append(nn.Linear(100, 10))
        self.layers = nn.Sequential(*layers)
        self.init_form = init_form
        if self.init_form is not None:
            self.init()

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = self.layers(x)
        return x

    # xavier weight initialization
    def init(self):
        sigmoid_gain = torch.nn.init.calculate_gain("sigmoid")
        for child in self.layers.children():
            if isinstance(child, nn.Linear):
                if self.init_form == "normal":
                    torch.nn.init.xavier_normal_(child.weight,
                                                 gain=sigmoid_gain)
                    if child.bias is not None:
                        torch.nn.init.zeros_(child.bias)
                elif self.init_form == "uniform":
                    torch.nn.init.xavier_uniform_(child.weight,
                                                  gain=sigmoid_gain)
                    if child.bias is not None:
                        torch.nn.init.zeros_(child.bias)
                else:
                    raise NotImplementedError()

Запустим обучение модели с инициализацией весов Xavier:

In [ ]:
model = SimpleMNIST_NN(n_layers=3).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.001)

# plotting weights values of first(input layer)
plt.figure(figsize=(15, 5))
plt.hist(list(model.layers.children())[0].weight.cpu().detach().numpy().reshape(-1), bins=100) 
plt.title('weights histogram')
plt.xlabel('values')
plt.ylabel('counts')
plt.show()

history = train_model(model, optimizer, model_name="n3_layers_sigmoid_havier")

In [ ]:
plot_history(history)

Видим, что нейросеть стала хоть как-то учиться

# Регуляризация

Второй способ улучшения сходимости нейросетей и борьбы с переобучением - введение регуляризации. Ее можно вводить несколькими способами. 

## L1, L2 регуляризации

Самый простой способ мы уже разбирали - давайте просто добавим в лосс штраф к весам. На сходимость нейросети, это, правда, повлияет слабо

$$Loss\_reg = loss + \lambda \cdot reg$$

$$ reg_{L1} = \lambda \sum |w_i| $$

$$ reg_{L2} = \lambda \sum w_i^2 $$

<img src="https://edunet.kea.su/repo/EduNet-content/L07/out/l1_l2_regularization_to_weight.gif" alt="alttext" width="500px"/>

[Visualizing regularization and the L1 and L2 norms](https://towardsdatascience.com/visualizing-regularization-and-the-l1-and-l2-norms-d962aa769932)

Иногда уже его хватает, чтобы решить все проблемы. Напомним, что L2 лосс приводит к большому числу маленьких ненулевых весов в сети. А L1 лосс - к маленькому числу ненулевых весов (разреженной нейросети)

## Dropout

Одним из распространенных именно в нейросетях методом регуляризации является Dropout.

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/out/dropout.png" width="700">

Состоит этот метод в следующем:

1. Во время обучения мы с вероятностью *p* зануляем выход нейронов слоя (например, *p* = 0.5)
2. Зануленные нейроны не участвуют в данном forward, и градиент потому к ним при backward не идет. 

3. Сила регуляризации определяется вероятностью p, чем она больше - тем сильнее регуляризация. 


### Мотивация Dropout

### Борьба с коадаптацией 

Одной из  проблем при работе с глубокими сетями является совместная адаптация нейронов. В такой сети, если все весовые коэффициенты изучаются вместе, обычно некоторые соединения будут иметь больше возможностей прогнозирования, чем другие.

Часть нейронов делает основную работу - предсказывает, а остальные могут вообще не вносить никакого вклада в итоговое предсказание. Или же другая картина - один нейрон делает неверное предсказание, другие его исправляют и в итоге первый нейрон свои ошибки не исправляет. 

Это явление называется коадаптацией. Этого нельзя было предотвратить с помощью традиционной регуляризации, такой как L1 и L2. А вот Dropout с этим хорошо борется

Отмена способности некоторых нейронов учиться во время тренировки на самом деле, оказывается, направлена на то, чтобы получить более обученные нейроны и сократить переобучение. И гарантирует, что выучиваемые индивидуальными нейронами функции хорошо работают со случайно выбранными подмножествами функций, выученных другими нейронами. 
Таким образом, Dropout приводит к появлению более требовательных нейронов, которые пытаются обойти сложные, индивидуальные особенности, которые склонны плохо обобщать, и сохраняют больше полезной информации самостоятельно. 

На следующем рисунке (извлеченном из статьи [Dropout: A Simple Way to Prevent Neural Networks from
Overfitting](https://jmlr.org/papers/v15/srivastava14a.html)) мы находим сравнение признаков, изученных в наборе данных MNIST с одним скрытым слоем в автоэнкодере, имеющим 256 признаков после ReLU без dropout (слева), и признаков, изученных той же структурой с использованием dropout в ее скрытом слое с p=0.5 (справа).

В то время как первый показывает неструктурированные, беспорядочные паттерны, которые невозможно интерпретировать, второй явно демонстрирует целенаправленное распределение веса, которое обнаруживает штрихи, края и пятна самостоятельно, нарушая их взаимозависимость с другими нейронами для выполнения этой работы.

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L07/compare_weights_with_dropout_and_without_dropout.png" width="600">

### Dropout как регуляризация 

Фактически, Dropout штрафует слишком сложные, неустойчивые решения. Добавляя в нейросеть Dropout, мы сообщаем ей о том, что решение, которое мы ожидаем, должно быть устойчиво к шуму

### Dropout как ансамбль 

Можно рассматривать Dropout как ансамбль нейросетей со схожими параметрами, которые мы учим одновременно, вместо того, чтобы учить каждую в отдельности, а затем результат их предсказания усредняем, [замораживая Dropout](https://prvnk10.medium.com/ensemble-methods-and-the-dropout-technique-95f36e4ae9be)

Фактически, возникает аналогия со случайным лесом - каждая из наших нейросетей легко выучивает выборку и переобучается - имеет низкий bias, но высокий variance. При этом за счет временного отключения активаций, каждая нейросеть видит не все объекты, а только часть. Усредняя все эти предсказания, мы уменьшаем variance. 



### Dropout помогает бороться с переобучением

Dropout, в силу указанного выше, может хорошо помогать бороться с переобучением

И в случае линейных слоев 

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L07/dropout_solve_overfitting_problem_in_mlp_networks.png" width="500">

[Tutorial: Dropout as Regularization and Bayesian Approximation](https://xuwd11.github.io/Dropout_Tutorial_in_PyTorch/)

И в случае свёрточных слоёв 

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L07/dropout_solve_overfitting_problem_in_convolution_networks.png" width="500">

### Confidence interval от Dropout
Можно используя нейросеть с дропаутом, получить доверительный интервал для нашего предсказания. (Как делали в лекции по ML) Просто не "замораживаем" dropout-слои во время предсказания, а делаем предсказания с активными dropout. 

И делаем forward через такую нейросеть для одного объекта 1000 раз. 
Сделав это 1000 раз, вы получаете распределение предсказаний, на основе которого можно делать confidence интервалы и как раз ловить те объекты, на которых нейросеть вообще не понимает, что ей делать и потому предсказывает метку или еще что-то с сильной дисперсией. 


<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L07/confidence_interval_dropout.png" width="600">



### Простая реализация Dropout

In [ ]:
class BadDropout(nn.Module):
    def __init__(self, p: float=0.5):
        super().__init__()
        if p < 0 or p > 1:
            raise ValueError(
                f"Dropout probability has to be between 0 and 1, but got {p}")
        self.p = p

    def forward(self, x):
        if self.training:
            keep = torch.rand(x.size()) > self.p
            if x.is_cuda:
                keep = keep.to(device)
            return x * keep
        # in test time, expectation is calculated
        return x * (1 - self.p)

Нежелательное свойство представленной выше схемы состоит в том, что мы должны масштабировать (сохранять среднее значение) активации во время прогнозирования. Поскольку производительность тестирования критически важна, лучше всего использовать инвертированный dropout, который выполняет масштабирование во время обучения. Кроме того, если мы захотим убрать dropout из кода, функция прогнозирования останется без изменений.

In [ ]:
class Dropout(nn.Module):
    def __init__(self, p: float=0.5):
        super().__init__()
        if p < 0 or p > 1:
            raise ValueError(
                f"Dropout probability has to be between 0 and 1, but got {p}")
        self.p = p

    def forward(self, x):
        if self.training:
            keep = torch.rand(x.size()) > self.p
            if x.is_cuda:
                keep = keep.to(device)
            return x * keep / (1 - self.p)
        return x  # in test time, expectation is calculated intrinsically - we just not divide weights

Попробуем применить Dropout в нашей нейросети:

In [ ]:
class SimpleMNIST_NN_Dropout(nn.Module):
    def __init__(self,
                 n_layers,
                 activation=nn.Sigmoid,
                 init_form="normal"):
        super().__init__()
        self.n_layers = n_layers
        self.activation = activation()
        layers = [nn.Linear(28 * 28, 100), self.activation]
        for _ in range(0, n_layers - 1):
            layers.append(nn.Linear(100, 100))
            layers.append(Dropout())           # add Dropout 
            layers.append(self.activation)
        layers.append(nn.Linear(100, 10))
        self.layers = nn.Sequential(*layers)
        self.init_form = init_form
        if self.init_form is not None:
            self.init()

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = self.layers(x)
        return x

    def init(self):
        sigmoid_gain = torch.nn.init.calculate_gain("sigmoid")
        for child in self.layers.children():
            if isinstance(child, nn.Linear):
                if self.init_form == "normal":
                    torch.nn.init.xavier_normal_(child.weight,
                                                 gain=sigmoid_gain)
                    if child.bias is not None:
                        torch.nn.init.zeros_(child.bias)
                elif self.init_form == "uniform":
                    torch.nn.init.xavier_uniform_(child.weight,
                                                  gain=sigmoid_gain)
                    if child.bias is not None:
                        torch.nn.init.zeros_(child.bias)
                else:
                    raise NotImplementedError()

Так как наша модель из-за Dropout ведет себя по-разному во время обучения и во время тестирования, то мы должны прямо ей сообщать, обучается она сейчас или нет. Делается это при помощи функций model.train и model.eval

In [ ]:
from tqdm import tqdm

def train_model_sep(model, optimizer, model_name=None, n_epochs=5):
  
    criterion = nn.CrossEntropyLoss().to(device)

    train_history = {}
    train_history['model_name'] = model_name
    train_history['loss_on_train'] = []
    train_history['loss_on_test'] = []

    for epoch in tqdm(range(n_epochs)):
        model.train()
        loss_on_train = train_epoch(model,
                                    optimizer,
                                    criterion,
                                    train_loader)
        model.eval()
        _, loss_on_test = validate(model,
                                   criterion,
                                   test_loader)
        train_history['loss_on_train'].extend(loss_on_train)
        train_history['loss_on_test'].extend(loss_on_test)
    return train_history

Обучим модель с Dropout:

In [ ]:
model = SimpleMNIST_NN_Dropout(n_layers=3).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.001)

history = train_model_sep(model, optimizer, model_name="nn3_dropout")
plot_history(history)

В данном случае выигрыша мы не получили. Возможно, если учить нейросеть больше эпох, эффект бы заметили

### Пример борьбы с переобучением при помощи Dropout
 

Чтобы увидеть эффект, и при этом не учить нейросеть 100+ эпох, сделаем искусственный пример.

Просто добавим к линейной зависимости шум и попробуем выучить ее нейронной сетью.

[Dropout in Neural Networks with Pytorch](https://towardsdatascience.com/batch-normalization-and-dropout-in-neural-networks-explained-with-pytorch-47d7a8459bcd)

In [ ]:
N = 50  # number of data points
noise = 0.3

# generate the train data
x_train = torch.unsqueeze(torch.linspace(-1, 1, N), 1)
y_train = x_train + noise * torch.normal(torch.zeros(N, 1), torch.ones(N, 1))

# generate the test data
x_test = torch.unsqueeze(torch.linspace(-1, 1, N), 1)
y_test = x_test + noise * torch.normal(torch.zeros(N, 1), torch.ones(N, 1))

print(f'x_train shape: {x_train.shape}\nx_test shape: {x_test.shape}')

In [ ]:
plt.scatter(x_train.data.numpy(), y_train.data.numpy(),
            c='purple', alpha=0.5, label='train')
plt.scatter(x_test.data.numpy(), y_test.data.numpy(),
            c='yellow', alpha=0.5, label='test')

x_real = np.arange(-1, 1, 0.01)
y_real = x_real
plt.plot(x_real, y_real, c="green", label='true')
plt.legend()
plt.show()

Модель без дропаут

In [ ]:
N_h = 100 # num of neurons
model = torch.nn.Sequential(
    torch.nn.Linear(1, N_h),
    torch.nn.ReLU(),
    torch.nn.Linear(N_h, N_h),
    torch.nn.ReLU(),
    torch.nn.Linear(N_h, 1),
)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

Модель с дропаутом

In [ ]:
N_h = 100 # num of neurons

model_dropout = nn.Sequential(
    nn.Linear(1, N_h),
    nn.Dropout(0.5),  # 50 % probability
    nn.ReLU(),
    torch.nn.Linear(N_h, N_h),
    nn.Dropout(0.2),  # 20% probability
    nn.ReLU(),
    torch.nn.Linear(N_h, 1),
)
optimizer_dropout = torch.optim.Adam(model_dropout.parameters(), lr=0.01)

In [ ]:
num_epochs = 1500
criterion = torch.nn.MSELoss()

for epoch in range(num_epochs):

    # train without dropout
    y_pred = model(x_train)  # look at the entire data in a single shot
    loss = criterion(y_pred, y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # train with dropout
    y_pred_dropout = model_dropout(x_train)
    loss_dropout = criterion(y_pred_dropout, y_train)
    optimizer_dropout.zero_grad()
    loss_dropout.backward()
    optimizer_dropout.step()

    if epoch % 100 == 0:

        model.eval() # not train mode
        model_dropout.eval() #  not train mode

        # get predictions
        y_test_pred = model(x_test)
        test_loss = criterion(y_test_pred, y_test)

        y_test_pred_dropout = model_dropout(x_test)
        test_loss_dropout = criterion(y_test_pred_dropout, y_test)
        # plotting data and predictions
        plt.scatter(x_train.data.numpy(), y_train.data.numpy(),
                    c='purple', alpha=0.5, label='train')
        plt.scatter(x_test.data.numpy(), y_test.data.numpy(),
                    c='yellow', alpha=0.5, label='test')
        plt.plot(x_test.data.numpy(), y_test_pred.data.numpy(),
                 'r-', lw=3, label='normal')
        plt.plot(x_test.data.numpy(), y_test_pred_dropout.data.numpy(),
                 'b--', lw=3,  label='dropout')

        plt.title('Epoch %d, Loss = %0.4f, Loss with dropout = %0.4f' %
                  (epoch, test_loss, test_loss_dropout))

        plt.legend()

        model.train() # train mode
        model_dropout.train() # train mode
 
        plt.pause(0.05)

Видим, что нейросеть без dropout сильно переобучилась.

## DropConnect

Если занулять не нейроны (активации), а случайные веса, с вероятностью $p$ получится DropConnect

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/out/dropconnect.png" width="650">

DropConnect похож на Dropout, поскольку он вводит динамическую разреженность в модель, но отличается тем, что разреженность зависит от весов *W*, а не от выходных векторов слоя. Другими словами, полностью связанный слой с DropConnect становится разреженно связанным слоем, в котором соединения выбираются случайным образом на этапе обучения. 

В принципе, вариантов зануления чего-то в нейросетке можно предложить великое множество, в разных ситуациях будут работать разные ([в этом списке](https://paperswithcode.com/methods/category/regularization)  много Drop...)

## DropBlock

Например, можно убирать для каждого батча из нейросети случайные блоки из слоев. И это будет работать!


<img src ="https://edunet.kea.su/repo/EduNet-content/L07/out/dropblock.png" width="750">

[Why Self-training with Noisy Students beats SOTA Image classification while using fewer resources](https://medium.datadriveninvestor.com/the-next-big-thing-in-image-classification-self-training-with-noisy-student-for-improving-22d52dc74dda)

# Нормализация

## Нормализация входных данных

Представим себе, что данные, которые мы подаем в нейросеть, распределены следующим образом

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/out/data_before_normalization.png" width="400">

Фактически нейросети работают со скалярными произведениями. В этом плане два вектора, изображенных на рисунке, не сильно отличаются. Так же и точки нашего датасета слабо разделимы. Чтобы с этим работать, нейросеть сначала должна подобрать удобное преобразование, а затем только сравнивать наши объекты. Понятно, что это усложняют задачу. 

Для того, чтобы облегчить нейросети задачу, входные признаки часто нормируют:

$$x1' = \dfrac {x1 - \mu_{x1}} {\sigma_{x1}}$$
$$x2' = \dfrac {x2 - \mu_{x2}} {\sigma_{x2}}$$

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/out/data_after_normalization.png" width="450">


 Такое преобразование действительно помогает нейросети 

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/out/normalization_helps_find_minimum_of_function.png" >


[deeplearning.ai](https://www.deeplearning.ai)

## Covariate shift (Ковариантный сдвиг)

Covariate shift - явление, когда признаки тренировочной выборки и тестовой по-разному распределены.

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/out/covariate_shift.png" width="300">

В такой ситуации модель не в состоянии делать адекватные предсказания на тесте, так как сеть не видела области пространства, в которой расположены тестовые объекты

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/out/covariate_shift_problem.png" width="300">

## Internal covariate shift

Похожее явление может иметь место уже внутри нейросети

Пусть у нас $i$-й слой переводит выход $i$-1 слоя в новое пространство. 

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/out/internal_covariate_shift_example_1_step.png" width="500">

В конце нейросеть делает предсказание, считается лосс, делается обратное распространение ошибки и обновляются веса. 

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/out/internal_covariate_shift_example_2_step.png" width="500">

После этого возникает нехорошая ситуация - распределение выходов $i$-1 слоя поменялось, а $i$-й слой изменял веса, думая, что распределение выходов не изменилось

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/out/internal_covariate_shift_example_3_step.png" width="500">

### Плохой вариант борьбы с этим

Давайте на каждом слое просто нормировать каждый признак, используя среднее и дисперсию по батчу

$$ \hat{x}_{i} = \frac{x_{i} - \mu_{B}}{\sigma_{B} + \epsilon}$$




Проблема в том, что таким образом мы можем попасть в область линейной составляющей нашей функции. Например, в случае сигмоиды 

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/out/domain_of_linear_of_sigmoid_function.png" width="500">

Получаем набор линейных слоев фактически без функций активации -> все вырождается в однослойную сеть. Не то, что нам надо

## BatchNormalization

Нам надо дать нейронной сети возможность перемещать распределение выходов слоя из области 0 и самой подбирать дисперсию. Для этой цели используется **батч-нормализация** (*batch normalization*), которая вводит в нейронную сеть дополнительную операцию между соседними скрытыми слоями. Она состоит из нормализации входящих (в слой батч-нормализации) значений, полученных от одного скрытого слоя, масштабирования и сдвига с применением двух новых параметров и передачи полученных значений на вход следующему скрытому слою.

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/out/batchnormalization.png" width="800">

Параметры, используемые в батч-нормализации $\gamma$ &mdash; отвечающий за сжатие и $\beta$ &mdash; отвечающий за сдвиг являются обучаемыми параметрами (наподобие весов и смещений скрытых слоев). 

Помимо обучаемых параметров $\gamma$ и $\beta$ в слое батч-нормализации существуют также необучаемые параметры: __скользящее среднее__ средних (_Mean Moving Average_) и скользящее среднее дисперсий (_Variance Moving Average_), служащие для сохранения состояния слоя батч-нормализации. 

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/out/batchnorm_layer_parameters.png" >  

Параметры $\gamma$, $\beta$, а также оба скользящих средних вычисляются для каждого слоя батч-нормализации отдельно и являются векторами с длиной, равной количеству входящих признаков.

В процессе обучения, мы подаем в нейронную сеть по одному мини-батчу за раз. Процедуру обработки значений $x$ из одного мини-батча $ B = \{x_{1},\ldots, x_{m}\} $ можно представить следующим образом:

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/out/batch_normalization_compute_moving_average.png" width="1000">

Шаг масштабирования и сдвига является главным новшеством батч-нормализации, поскольку в отличие от предыдущего рассмотренного примера, нормированные значения больше не обязаны иметь среднее равное 0 и единичную дисперсию. Батч-нормализации позволяет сдвигать среднее нормированных значений и масштабировать дисперсию. Фактически, теперь нейросеть даже может отменить нормализацию входных данных, если считает ее ненужной.

Для наглядности проиллюстрируем размерности промежуточных переменных на следующем изображении:

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/out/batch_normalization_compute_moving_average_scheme.png" width="1000">

После прямого прохода, параметры $\gamma$ и $\beta$ обновляются через обратное распространение ошибки также как и веса скрытых слоев.

### Скользящее среднее

Выше мы обсуждали то, что в процессе обучения слой  Батч-нормализация рассчитывает значение среднего и дисперсии каждого признака в соответствующем мини-батче. Однако во время предсказания, батч у нас уже отсутствует &mdash; откуда брать среднее и дисперсию? 

Во время предсказания используется скользящее среднее, которое было рассчитано и сохранено в процессе обучения. 

$$ \mu_{mov_{B}} = (1-\alpha)\mu_{mov_{B-1}}+\alpha\mu_{B} $$

$$ \sigma_{mov_{B}} = (1-\alpha)\sigma_{mov_{B-1}}+\alpha\sigma_{B} $$

Обычно используется параметр $\alpha = 0.1$

Почему используется именно скользящее среднее, а не, например, статистика всей обучающей выборки? Дело в том, что при таком подходе нам бы пришлось хранить средние всех признаков для всех батчей, пропущенных через нейросеть в ходе обучения. Это ужасно невыгодно по памяти. Вместо этого скользящее среднее выступает в качестве приближенной оценки среднего и дисперсии обучающего набора, в этом случае эффективность использования ресурсов увеличивается, поскольку вычисления производятся постепенно &mdash; нам нужно хранить в памяти только одно число &mdash; значение скользящего среднего, полученное на последнем шаге.

Проиллюстрировать преимущество использования скользящего среднего можно на следующем примере:

Предположим, что у нас есть некоторая генеральная совокупность объектов, обладающих некоторым признаком $x$, (соответствующая абстрактной обучающей выборке) и некоторый черный ящик, извлекающий по $k$ объектов из этой генеральной совокупности (соответствующий абстрактному даталоадеру). Наша задача &mdash; дать оценку ожидаемому среднему этих $k$ объектов. При этом ожидаемое среднее для $k$ объектов может отличаться от среднего генеральной совокупности, поскольку могут накладываться дополнительные условия на извлекаемую выборку. В данном примере для простоты будем извлекать $k$ объектов из некого распределения случайным образом:

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

k = 500 # sample size
n = 2
p = 0.5

sample = np.random.negative_binomial(n, p, k)
sns.histplot(data=sample, discrete=True)
plt.show()

Оценить ожидаемое среднее теоретически, не зная как распределен признак $x$ наших объектов, трудно. Мы можем собрать большое количество средних и произвести оценку с их помощью, но для этого нам потребуется хранить в памяти все эти значения, что опять-таки приведет к неэффективному расходу ресурсов. Более эффективным решением будет воспользоваться скользящим средним. Давайте сравним эти два метода:

In [ ]:
ema = 0
alpha = 0.01
means = np.array([])

for i in range(10000):
    sample = np.random.negative_binomial(n, p, 50)
    ema = (1 - alpha) * ema + alpha * sample.mean()
    means = np.append(means, sample.mean())

Посчитаем количество памяти, затрачиваемое на хранение списка средних значений признака $x$ по выборкам из $k$ объектов, и количество памяти, затрачиваемое на хранение скользящего среднего:

In [ ]:
import sys

print(f'{sys.getsizeof(ema)} bytes')

Количество памяти для хранения списка средних:

In [ ]:
print(f'{sys.getsizeof(means)} bytes')

Видно, что на хранение массива средних значений расходуется на порядки больше памяти, чем на хранение одного скользящего среднего. Теперь давайте воспользуемся тем, что мы сэмплировали случайные выборки из известного распределения, и можем теоретически рассчитать их среднее. В нашем примере мы извлекали выборки из негативного биномиального распределения с параметрами $n=2$ и $p=0.5$, для которого среднее рассчитывается по формуле $mean=\frac{np}{1-p}=2$. Мы знаем, что при достаточно большом количестве сэмплированных выборок, среднее распределения выборочных средних будет стремиться к среднему генеральной совокупности. Сравним результаты, полученные с использованием сохраненных выборочных средних и скользящего среднего с теоретическим расчетом:

Среднее признака x по k объектам, оцененное с помощью скользящего среднего:

In [ ]:
print(f'{ema:.8f}')

Среднее признака x по k объектам, оцененное по всем сэмплированным выборкам:

In [ ]:
print(f'{means.mean():.8f}')

Видно, что мы получили довольно точную оценку, использовав скользящее среднее.

### Защита от нулей в знаменателе

Чтобы у нас не мог возникнуть 0 в знаменателе, добавляем маленькое число - $\epsilon$. Например, равное 1e-5


$$ \hat{x}_{i} = \frac{x_{i} - \mu_{B}}{\sigma_{B} + \epsilon}$$

$$ BN_{\gamma, \beta}(x_{i}) = \gamma \hat{x}_{i} + \beta $$

### Линейный слои и конволюции

Слой конволюции можно свести к линейному слою с очень жесткими ограничениями на веса. Поэтому неудивительно, что BatchNorm можно применять и для линейных слоев и для конволюций. 

С конволюциями есть единственный нюанс - у нас "одним признаком" считается вся получаемая **feature map**. 

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/out/feature_map.png" width="500">

И нормализация идет по всей такой feature map (по всему каналу) для всех объектов. 

### Пример работы

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L07/batchnorm_efficiency.png" width="550">


Этот метод действительно работает. 
Видим, что нейросети с батч-нормализацией:

1. Сходятся быстрее, чем нейросети без неё
2. Могут работать с более высоким начальным learning rate, причем это позволяет достигать лучших результатов
3. BatchNorm позволяет глубокой нейросетке работать даже с функцией активации в виде сигмоиды. Без батчнорма такая сеть не обучилась бы вовсе. 

### Градиент

Вычисление градиента batchnorm - интересное упражнение на понимание того, как работает backpropagation. В лекции мы это опускаем, можете ознакомиться самостоятельно

[Deriving the Gradient for the Backward Pass of Batch Normalization](https://kevinzakka.github.io/2016/09/14/batch_normalization/)



<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L07/batchnorm_gradient.png" width="700">

### Batchnorm как регуляризация

Почему для нейросети с батчнормализацией можно использовать более высокие learning rate? 

 

Оказывается, батчнормализация делает неявную регуляризацию на веса.

Допустим, мы решили увеличить веса в $a$ раз

Так как мы шкалируем, то домножение весов $W$ на константу выходных значений слоя не меняет

$$BN((aW)u) = BN(Wu)$$

Градиент слоя по входу не меняется

$$\dfrac {\delta BN((aW)u)} {\delta u} = \dfrac {\delta BN(Wu)} {\delta u}$$

А градиент по весам уменьшается в $a$ раз

$$\dfrac {\delta BN((aW)u)} {\delta aW} = \dfrac 1 a \dfrac {\delta BN(Wu)} {\delta W} $$

Таким образом, нейросеть автоматически не дает большим весам расти

### Internal covariate shift?

Согласно некоторым исследованиям ([например](https://arxiv.org/abs/1805.11604)), успех BatchNormalization заключается не в исправлении covariate shift. BatchNormalization работает как-то иначе, улучшая гладкость пространства решений и облегчает поиск в нем минимума.

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/out/batchnorm_helps_find_minimum_of_function.jpg" width="900">

### Советы по использованию BatchNormalization

Стоит помнить, что с батч-нормализацией:

* **Крайне важно** перемешивать объекты (составлять новые батчи) между эпохами. Единицей обучения параметров $\beta$ и $\gamma$ являются батчи. Если их не перемешивать, то из 6400 объектов в тренировочном датасете получим лишь 100 объектов (при условии, что в батче 64 объекта) для обучения $\beta$ и $\gamma$

* В слое, после которого поставили BatchNormalization, надо убрать bias (параметр $\beta$ в BatchNormalization берет эту роль сам по себе)


* Другое расписание learning rate: бОльшее значение в начале обучения и быстрое уменьшение в процессе обучения

* Если используем BatchNormalization, то надо уменьшить силу Dropout и L2-регуляризации

* Чем меньше размер батча в обучении, тем хуже будет работать BatchNormalization

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L07/batchnorm_batch_size.png" width="550">



### Используем BatchNormalization в Pytorch

In [ ]:
class SimpleMNIST_NN_Init_Batchnorm(nn.Module):
    def __init__(self, n_layers):
        super().__init__()
        self.n_layers = n_layers
        layers = [nn.Linear(28 * 28, 100, bias=False),
                  nn.BatchNorm1d(100),
                  nn.Sigmoid()]
        for _ in range(0, n_layers - 1):
            layers.append(nn.Linear(100, 100, bias=False))
            layers.append(nn.BatchNorm1d(100))
            layers.append(nn.Sigmoid())
        layers.append(nn.Linear(100, 10))
        self.layers = nn.Sequential(*layers)
        self.init()

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = self.layers(x)
        return x

    def init(self):
        sigmoid_gain = torch.nn.init.calculate_gain("sigmoid")
        for child in self.layers.children():
            if isinstance(child, nn.Linear):
                torch.nn.init.xavier_normal_(child.weight,
                                             gain=sigmoid_gain)
                if child.bias is not None:
                    torch.nn.init.zeros_(child.bias)

In [ ]:
model = SimpleMNIST_NN_Init_Batchnorm(n_layers=3).to(device)
optimizer = optim.SGD(model.parameters(), lr=1e-4)

hooks_data_history = register_model_hooks(model)
history = train_model_sep(model, optimizer, model_name="batchnorm2")
plot_history(history)

In [ ]:
plot_hooks_data(hooks_data_history)

И попробуем, согласно советам, увеличить learning rate

In [ ]:
model = SimpleMNIST_NN_Init_Batchnorm(n_layers=3).to(device)
optimizer = optim.SGD(model.parameters(), lr=1e-2)

hooks_data_history = register_model_hooks(model)
history = train_model_sep(model, optimizer, model_name="batchnorm_increased_lr")
plot_history(history)

In [ ]:
plot_hooks_data(hooks_data_history)

### Ставить BatchNormalization до или после активации?



#### До


<img src ="https://edunet.kea.su/repo/EduNet-content/L07/out/batchnormalization_before_activation.png" width="400">

* Рекомендуется авторами статьи, где предложили Batch Normalization
* Для сигмоиды, BN, поставленная после активации, не решает проблем сигмоиды

#### После

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/out/batchnormalization_after_activation.png" width="400">

* Аргументация авторов статьи не до конца обоснована
* Обычно, сигмоиду не используют в современных нейронных сетях
* Для популярной ReLU, BN, поставленная до активации может приводить к “умирающей ReLU”, когда большая часть ее входов меньше 0 и потому для них градиент не проходит
* На многих задачах BN после функции активации работает лучше или не хуже поставленной до

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L07/batchnormalization_before_or_after_relu.png" width="500">

[BN experiments](https://github.com/ducha-aiki/caffenet-benchmark/blob/master/batchnorm.md)

### Ставить BatchNormalization до или после Dropout?



#### До

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/out/batchnormalization_before_dropout.png" width="600">

* Меньше влияние (covariate shift) Dropout на Batchnorm

#### После

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/out/batchnormalization_after_dropout.png" width="600">

* Информация о зануленных активациях не просачивается через среднее и дисперсию батча

### Ставить только что-то одно 

* Dropout может отрицательно влиять на качество нейросети с BatchNorm за счет разного поведения на train и test




### Строго говоря

* Оптимальный порядок следования слоев зависит от задачи и архитектуры сети
* Возможно, стоит применять модифицированные версии BatchNorm

## Другие Normalization

Существует большое количество иных нормализаций, их неполный список можно найти [здесь](https://paperswithcode.com/methods/category/normalization). В данной секции мы рассмотрим самые популярные виды нормализации помимо BatchNorm.

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L07/normalization_methods.png" width="900">

Чаще всего нормализации тестируют на изображениях, в том числе или только на изображениях. Потому их графически объясняют при помощи следующей схемы:

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/out/dimensions_channels_batch_samples.png" width="450">

*По оси абсцисс* расположены объекты из батча,  
*по оси ординат* - feature map, преобразованный в вектор,  
*по оси аппликат* - каналы (feature maps).

В этом представлении BatchNorm выглядит следующим образом:

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/out/visualization_of_batch_normalization.png" width="450">

[Batch Normalization](https://paperswithcode.com/method/batch-normalization)

### Layer Norm

Помимо свёрточных нейронных сетей, существует специальный тип нейронных сетей, используемых для обработки последовательностей. Называется он "рекуррентные нейронные сети", ему же будет посвящена наша следующая лекция. 

Когда оказалось, что BatchNorm положительно сказывается на обучении нейронных сетей, его попытались применить для различных архитектур. BatchNorm нельзя было использовать "из коробки" для рекуррентных нейронных сетей (работающих с последовательными данными), пришлось придумывать различные адаптации, среди которых наиболее удачной оказалась **Layer Normalization**.  
По сути, теперь нормализация происходит внутри одного объекта из батча, а не поканально в рамках батча. С математической точки зрения, данная "адаптация" отличается от BatchNorm, однако экспериментально она превзошла своих конкурентов в задаче нормализации при обработке последовательных данных.

Впоследствии, данный метод нормализации хорошо проявил себя в трансформерах - наследниках рекуррентных нейронных сетей в вопросах обработки последовательных данных (об этом типе нейросетей мы также поговорим на следующей лекции). После успешного применения трансформеров в задачах компьютерного зрения, LayerNorm стал использоваться и в компьютерном зрении (хотя и уступал BatchNorm при использовании в свёрточных нейронных сетях).

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/out/visualization_of_layer_normalization.png" width="450">

[Layer Normalization](https://paperswithcode.com/method/layer-normalization)


### Instance Norm

Следующий вид нормализации был предложен отечественными исследователями (из Сколтеха), занимавшимися разработкой быстрого и эффективного способа переноса стиля с одного изображения на другое. 

При использовании BatchNorm терялась информация о контрастах на конкретном изображении, поскольку нормализация производится по нескольким объектам. Для сохранения *контрастов* в экземпляре (*instance*) изображения была предложена специальная нормализация, рассматривающая конкретный канал одного конкретного объекта. Было предложено два названия нормализации: связанное с мотивацией (contrast normalization) и связанное с принципом работы (**instance normalization**). 

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/out/visualization_of_instance_normalization.png" width="450">

[Instance Normalization](https://paperswithcode.com/method/instance-normalization)

### Group Norm

В течение долгого времени, BatchNorm оставался однозначным фаворитом для использования в задачах компьютерного зрения, однако:
1. В связи с необходимостью точно считать статистики внутри batch, при обучении приходилось использовать большой batch size.  

2. Ограниченность размера видеопамяти вынуждает разработчиков идти на компромисс между сложностью модели и batch size.

Итого, использование BatchNorm приводило к невозможности использовать сложные модели**\***, поскольку им просто не хватало места на видеокарте. 

Необходимость использовать большой batch size могут решать различные нормализации, не использующие batch-размерность. К примеру, уже известные нам **Layer Norm** и **Instance Norm**. Эмпирически оказалось, что данные нормализации уступают BatchNorm по качеству работы: в то время как LayerNorm предполагает одинаковую важность и суть различных каналов (*рассматривая данные излишне глобально*), InstNorm упускает межканальные взаимодействия (*рассматривая данные слишком локально*). 

Успешным обобщением данных методов является **Group Normalization**: данный метод разбивает каналы на $G \in [1; C]$ групп, присваивая каждой из них (примерно) равное количество каналов. Отметим, что при $G = 1$, GroupNorm идентичен LayerNorm, а при $G = C$, GroupNorm идентичен InstNorm. 

Эмпирически оказалось, что при замене BatchNorm на GroupNorm качество модели падает в разы менее значительно, чем при использовании LayerNorm либо GroupNorm. Более того, при изменении batch size качество работы LayerNorm не изменялось, что открыло перспективы для создания более сложных моделей компьютерного зрения. 


**\*** - подразумевается, что уменьшение batch size позволило бы создать более сложные модели.

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/out/visualization_of_group_normalization.png" width="450">

[Group Normalization](https://paperswithcode.com/method/group-normalization)

### Weight Normalization

Существует семейство методов, принципиально отличающихся от рассмотренных ранее нормализаций: в отличие от BatchNorm, LayerNorm и прочих известных вам нормализаций, ориентированных на **изменение представлений данных** внутри нейронной сети (и по сути представляемых как обычные слои), некоторые методы повышения качества обучения нейронных сетей ориентированы на **изменение параметров** самой нейронной сети.

[Weight Normalization](https://paperswithcode.com/method/weight-normalization) - один из таких методов, во многом вдохновлённый BatchNorm. В отличие от ранее упомянутых нормализаций, он появился в результате анализа влияния BatchNorm на обучение с сугубо математической точки зрения (было рассмотрено влияние BatchNorm на [информационную матрицу Фишера](https://en.wikipedia.org/wiki/Fisher_information)).  



В данном методе производится репараметризация слоя нейронной сети следующим образом:

$$\mathbb{w} = \frac{g}{\left\Vert \mathbb{v}\right\Vert}\mathbb{v},$$

где $\mathbb{w}$ - вектор весов слоя, $g$ и $\mathbb{v}$ - обучаемые параметры. Важный момент, который стоит сразу отметить: данный вид параметризации не зависит от данных, потому одинаково хорошо будет работать при любом batch size. 

**Weight normalization** в некоторых случаях позволяет с лёгкостью выполнять ранее обсуждаемую "умную инициализацию" параметров нейронной сети. Выполняется она следующим образом: некоторым образом инициализируем $\mathbb{v}$; выполним запуск сети на случайном batch, пусть на вход слою приходит $\mathbb{x}$. Выполним следующую операцию:

$$t = \frac{\mathbb{v}\cdot\mathbb{x}}{\left\Vert\mathbb{x}\right\Vert}, \qquad y = \phi\left(\frac{t - \mu[t]}{\sigma[t]}\right)$$

Чтобы перед активацией получить значения с нулевым средним и единичным стандартным отклонением, стоит установить параметрам $g$ и $b$ следующие значения:

$$g \leftarrow \frac{1}{\sigma[t]}, \qquad b \leftarrow \frac{-\mu[t]}{\sigma[t]}.$$

Другой важной особенностью **weight normalization** является *разделение* вектора весов на "направление вектора" и "норму вектора". По сути, отделение нормы векторов весов позволяет реализовывать гибкую настройку скорости обучения - различные параметры начинают обновляться с различной скоростью ($\sim\lambda \cdot g_i$), что позволяет модели более гибко обучаться. 

Weight normalization - далеко не единственный метод, позволяющий улучшать обучение модели, влияя на обновление её параметров: существует целое семейство "оптимизаторов" - методов, отвечающих за изменение параметров обучаемой модели.

# Оптимизация весов нейросетей

Методов тоже много, расскажем о популярных ([неполный список](https://paperswithcode.com/methods/category/stochastic-optimization))

Эти методы реализованы в модуле torch.optim
При этом, что важно, подсчет градиентов лосс функции по весам никак не зависит от оптимизаторов из пакета. На прошлых занятиях мы с вами видели, что pytorch вычисляет градиенты автоматически, на основе оптимизаций.


## Обзор популярных оптимизаторов

### SGD
Обычный стохастичный градиентный спуск. Обновляем веса в соответствии с текущим градиентом по ним, домножая антиградиент на постоянный коэффициент $\eta$ (гиперпараметр обучения - learning rate).

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/out/stochastic_gradient_descent.gif" width="950">

SGD обладает важной особенностью - на каждом объекте или наборе объектов (мини-батче) ландшафт функции потерь выглядит по-разному. Некоторые минимумы функции потерь и седловые точки могут быть характерны лишь для части объектов.

Использование SGD позволяет модели при обучении выбираться из **некоторых** локальных минимумов. В результате обучения модель с большой вероятностью остановится в относительно широком и глубоком минимуме, хотя нет гарантий, что он будет "лучшим".

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/out/sgd_loss_batch_landscape.png" width="700">

SGD до сих пор является достаточно популярным методом обучения нейросетей, потому что он простой, не требует подбора дополнительных гиперпараметров кроме $\eta$ и сам по себе обычно дает неплохие результаты. 

Если же модель учится слишком долго и/или важна каждая сотая в качестве, то нужно либо использовать его вкупе с другими технологиями (о них - ниже), либо использовать другие способы.

In [ ]:
parameters = torch.randn(10, requires_grad=True)
optimizer = optim.SGD([parameters], lr=0.001)

Минусы SGD:

 1. Если функция ошибки быстро меняется в одном направлении и медленно - в другом, то это приводит к резким изменениям направления  градиентов и замедляет процесс обучения


<img src ="https://edunet.kea.su/repo/EduNet-content/L07/out/stohastic_gradient_descent_no_momentum.gif" width="500">


[Machine Learning Optimization Methods “Mechanics, Pros and Cons”](https://salmenzouari.medium.com/machine-learning-optimization-methods-mechanics-pros-and-cons-81b720194292)

 2. Может застревать в локальных минимумах или седловых точках (точках, где все производные равны 0, но не являющихся минимума/ максимумами). В них градиент равен 0, веса не обновляются - конец оптимизации. 

Пример таких точек:


Точка 0 у функции $x^3$, не имеющей минимума или максимума вовсе

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/out/getting_stuck_in_local_minimum_example.png" width="400">


Или точка 0, 0 у функции z = $x^2 - y^2$


<img src ="https://edunet.kea.su/repo/EduNet-content/L07/out/saddle_point_example.png" width="400">


[Седловая точка
](https://ru.wikipedia.org/wiki/%D0%A1%D0%B5%D0%B4%D0%BB%D0%BE%D0%B2%D0%B0%D1%8F_%D1%82%D0%BE%D1%87%D0%BA%D0%B0)

 3. Так как мы оцениваем градиенты по малой части выборки, они могут плохо отображать градиент по всей выборке и являться шумными. В результате часть шагов градиентного спуска делаются впустую или во вред. 
 
 4. Мы применяем один и тот же learning rate ко всем параметрам, что не всегда разумно. Параметр, отвечающий редкому классу, будет обучаться медленнее остальных. 
 
 5. Просто медленнее сходится

Основой всех описанных ниже алгоритмов является SGD.

### Momentum

Чтобы избежать проблем 1-3, можно использовать momentum - фактически, мы добавляем нашему движению инерции. Если представить наши текущие веса как координаты шарика, и мы этот шарик пытаемся загнать в наиболее глубокое место, то у шарика теперь появился вес.

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/out/stohastic_gradient_descent_with_momentum.gif" width="600">

[Градиентный спуск, как учатся нейронные сети(Видео)](https://youtu.be/IHZwWFHWa-w)

Сначала пытаемся поменять направление движения шарика с прежнего направления с учетом текущего градиента
$$v_{t+1} = \rho v_t + \nabla_wL(x, y, W)$$

Вычисляем, куда он покатится
$$w_{t+1} = w_t - \eta v_{t+1}$$

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/out/advantages_wtih_momentum.png" width="480">

In [ ]:
parameters = torch.randn(10, requires_grad=True)
optimizer = optim.SGD([parameters], momentum=0.9, lr=0.001)

Теперь мы быстрее достигаем локального минимума и можем выкатываться из совсем неглубоких. Градиент стал менее подвержен шуму, меньше осциллирует

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/out/stohastic_gradient_descent_no_momentum.gif" width="400">

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/out/stohastic_gradient_descent_with_momentum.gif" width="400">

У этого подхода есть одна опасность - мы можем выкатиться за пределы минимума, к которому стремимся, а потом какое-то время к нему возвращаться. 

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L07/problem_of_big_momentum_value.gif" width="700">

[optimizer-visualization](https://github.com/Jaewan-Yun/optimizer-visualization)

Чтобы с этим бороться, предложен другой способ подсчета момента инерции

### NAG (Nesterov momentum)

Будем сначала смещаться в сторону, куда привел бы нас наш накопленный градиент, там считать новый градиент и смещаться по нему. 
В результате перескоки через минимум будут менее значительными, и мы будем быстрее сходиться

$$v_{t+1} = \rho v_t +  \nabla_w L(w + \rho v_t )$$

$$w_{t} = w_{t-1} - \eta v_{t} $$


<img src ="https://edunet.kea.su/repo/EduNet-content/L07/out/nesterov_momentum.png" width="800">

На практике эту формулу все равно можно записать так, чтобы задача вычисления градиента ложилась не на сам оптимизатор
(к примеру, [реализация в PyTorch](https://github.com/pytorch/pytorch/blob/master/torch/optim/_functional.py))

In [ ]:
parameters = torch.randn(10, requires_grad=True)
optimizer = optim.SGD([parameters], momentum=0.9, nesterov=True, lr=0.001)

### Adaptive Learning Rate

Оба озвученных улучшения никак не борются с тем, что некоторые параметры нашей нейросети могут получать информацию для обновления реже, чем остальные. Например, фильтр, отличающий сиамских кошек от остальных, будет работать только тогда, когда в выборке попадутся конкретно сиамские кошки. А их может быть и не очень много, по батчам мы их не размажем (либо батчей получится очень много, если нарочно балансировать)

Другая ситуация - учим нейросеть на словах из русского языка. Есть веса, отвечающие за редкие слова, например "молвить". Вы часто в языке встречаете слово молвить? Молвят устами, а они тоже встречаются не часто. В результате, если learning rate постоянен, то мы выучим параметры для слова молвить плохо. 

Единственный путь - завести для каждого параметра индивидуальный learning rate.

### Adagrad

Будем хранить для каждого параметра сумму квадратов его градиентов. 

И будем добавлять наш градиент с коэффициентом, обратно пропорциональным корню из этой суммы (фактически - некая аппроксимация средней амплитуды градиента нашего параметра). 

$$ w = w - \eta \frac{l}{\sqrt{G} + e} \odot (\nabla_w L(x,y,W)) $$

$$ G = \sum_{t=1}^T \nabla_w L(x,y,w_t)^2 $$

В результате, если градиент у нашего веса часто большой - коэффициент будет уменьшаться. 

Единственная проблема - при такой формуле наш learning rate неминуемо в конце концов затухает (так как сумма квадратов не убывает)


In [ ]:
parameters = torch.randn(10, requires_grad=True)
optimizer = optim.Adagrad([parameters])

### RMSprop

Давайте устроим "забывание" предыдущих квадратов градиентов. Просто будем домножать их на некий коэффициент меньше 1


$$v_t = \alpha v_{t-1} + (1-\alpha) (\nabla_w L(x,y,w_t))^2$$

$$w = w - \frac{\eta}{\sqrt{v_t }+ e} \odot \nabla_w L(x,y,W)$$

In [ ]:
parameters = torch.randn(10, requires_grad=True)
optimizer = optim.RMSprop([parameters], alpha=0.99)

### Adam

Одним из самых популярных адаптивных оптимизаторов является Adam. 
Получается он за счет объединения идеи с инерцией и идеи с суммой квадратов. 

$$ m_t = \beta_1 m_{t-1} + (1-\beta_1) (\nabla_w L(x,y,w_t)) $$
$$ v_t = \beta_2 v_{t-1} + (1-\beta_2) (\nabla_w L(x,y,w_t)^2) $$
$$ w = w - \eta \cdot \frac{m_t}{\sqrt{v_t} + e} $$

Чтобы в начале у нас получались очень большие шаги, будем дополнительно модицифировать инерцию и сумму квадратов

$$ m_t = \frac{m_t}{1-\beta_1^t} $$

$$ v_t = \frac{v_t}{1-\beta_2^t} $$

In [ ]:
parameters = torch.randn(10, requires_grad=True)
optimizer = optim.Adam([parameters], betas=(0.9, 0.999))

In [ ]:
model = SimpleMNIST_NN_Init_Batchnorm(n_layers=3).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-2)
hooks_data_history = register_model_hooks(model)


history = train_model_sep(model, optimizer, model_name="adam")
plot_history(history)

In [ ]:
plot_hooks_data(hooks_data_history)

#### L2-loss

Все оптимизаторы так же поддерживают возможность добавления к ним напрямую L2-loss, коэффициент перед этим лоссом -  $\textrm{weight_decay}$

In [ ]:
parameters = torch.randn(10, requires_grad=True)
optimizer = optim.RMSprop([parameters], alpha=0.99, weight_decay=0.001)

Нюанс в том, что в Adam L2-loss учитывается не совсем верно. Потому есть поправленная версия Adam - AdamW. Но не факт, что она всегда лучше работает

## Сравнение оптимизаторов 

У каждого из предложенных оптимизаторов есть минусы и плюсы 



### Методы с инерцией сходятся к решению более плавно, но могут "перелетать"

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L07/convergence_optimizers.gif" width="250">



### Методы с адаптивным learning rate быстрее сходятся, более стабильны и меньше случайно блуждают

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L07/methods_with_adaptive_learning_rate.gif" width="250">


###  Алгоритмы без адаптивного learning rate сложнее выбираются из локальных минимумом

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L07/methods_without_adaptive_learning_rate.gif" width="450">

### Алгоритмы с инерцией осцилируют в седловых точках прежде чем найти верный путь

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L07/methods_with_momentum_in_saddle_point.gif" width="450">

<p><em>Иллюстрации <a href="https://imgur.com/a/Hqolp">Alec Radford</a></p> </em>


# Режимы обучения

Нам не обязательно поддерживать один и тот же learning rate в течение всего обучения. Более того, для того же SGD есть гарантии, что если правильно подобрать схему уменьшения learning rate, он сойдется к глобальному оптимуму.


Мы можем менять learning rate по некоторым правилам.  



## Ранняя остановка

Можем использовать критерий ранней остановки - когда лосс на валидационной выборке не улучшается какое-то количество эпох(patience), умножаем learning rate на некое значение(factor)

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/out/early_stopping.png" width="500">

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=0.1)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer,
                                                 'min',
                                                 factor=0.1,
                                                 patience=5)

Применим к нашей модели

(выполнение занимает ~ 5 минут)

In [ ]:
def train_model_sep_scheduler(model, optimizer, scheduler, model_name=None, n_epochs=5):
  
    criterion = nn.CrossEntropyLoss().to(device)

    train_history = {}
    train_history['model_name'] = model_name
    train_history['loss_on_train'] = []
    train_history['loss_on_test'] = []

    for epoch in tqdm(range(n_epochs)):
        model.train()
        loss_on_train = train_epoch(model,
                                    optimizer,
                                    criterion,
                                    train_loader)
        model.eval()
        val_loss, loss_on_test = validate(model,
                                   criterion,
                                   test_loader)
        train_history['loss_on_train'].extend(loss_on_train)
        train_history['loss_on_test'].extend(loss_on_test)
        scheduler.step(val_loss)
    return train_history


In [ ]:
model = SimpleMNIST_NN_Init_Batchnorm(n_layers=3).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-2)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer,
                                                 'min',
                                                 factor=0.1,
                                                 patience=1)


hooks_data_history = register_model_hooks(model)
history = train_model_sep_scheduler(model,
                                    optimizer,
                                    scheduler,
                                    model_name="reduce_lr_on_plateu",
                                    n_epochs=15)
plot_history(history, n_epochs=15)

## Понижение шага обучения на каждой эпохе 

Домножать learning rate на gamma каждую эпоху

In [ ]:
scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                            step_size=2,
                                            gamma=0.1)

## Cyclical learning schedule


Мы можем не все время понижать learning rate, а делать это циклически - то понижать, то повышать. Делать это можно по-разному:


1. Постоянно оставлять одни и те же границы

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/out/cyclical_learning_schedule_permanent_confines.png" width="600">


2. Уменьшать верхнюю границу во сколько-то раз

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/out/cyclical_learning_schedule_reduce_confines.png" width="600">

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/out/cyclical_learning_schedule_reduce_confines_smooth.png" width="600">

Нюанс - здесь мы ОБЯЗАТЕЛЬНО должны хранить модель с лучшим качеством. Такая оптимизация не гарантирует, что в конце модель будет лучше, чем на каком-то промежуточном шаге

### Подбираем границы learning rate



1. Просто берем и либо делаем для каждого learning rate минимизацию одну эпоху (чтобы всю выборку увидеть), нейросетку для каждого значения learning rate инициализируем заново

Это долго

Потому  делаем "магию":

2. На каждое значение learning rate у нас будет лишь один батч, нейросетку для каждого learning rate не меняем. И это часто работает, так как нам нужна просто грубая прикидка

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L07/optimal_learning_rate_range.png" width="650">

[Transfer Learning In NLP](https://medium.com/modern-nlp/transfer-learning-in-nlp-f5035cc3f62f)

Полученные значения (оптимальные границы)  используем в качестве высокого и низкого порога на learning rate.

In [ ]:
start_lr = 1e-8
end_lr = 10
lr_find_epochs = 2
steps = lr_find_epochs * len(train_loader)
smoothing = 0.05

In [ ]:
model = SimpleMNIST_NN_Init_Batchnorm(n_layers=3).to(device)
criterion = nn.CrossEntropyLoss().to(device)

In [ ]:
import math 

lrs = []
losses = []
optimizer = optim.SGD(model.parameters(), lr=1e-8)
lr_lambda = lambda x: math.exp(x * math.log(end_lr / start_lr) / (steps))

scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)

for epoch in tqdm(range(lr_find_epochs)):
    for batch in train_loader:
        optimizer.zero_grad()
        x_train, y_train = batch
        x_train, y_train = x_train.to(device), y_train.to(device)
        y_pred = model(x_train)
        loss = criterion(y_pred, y_train)

        loss.backward()
        optimizer.step()
        scheduler.step()

        loss = loss.detach().cpu().numpy()
        if len(losses) > 1:
            loss = smoothing * loss + (1 - smoothing) * losses[-1]
        losses.append(loss)
        lr_step = optimizer.state_dict()["param_groups"][0]["lr"]
        lrs.append(lr_step)

In [ ]:
plt.figure(figsize=(20, 10))
plt.title("LR range selection", size=15)
plt.plot(np.log10(lrs), losses)
plt.xlabel("Learning rate (log10)",size=15)
plt.ylabel("Loss")
plt.show()

На этом графике нам нужен минимум. Это в районе 1e-1. Делим это число на 10. 
Нижняя граница, стало быть - 1e-2.

Снижение лосса начинается с learning rate в районе 1e-4. Его делим на 6

In [ ]:
min_lr = 1e-4 / 6
max_lr = 1e-2

Этот scheduler надо применять после каждого батча. Потому перепишем train_epoch_sh

In [ ]:
def train_epoch_sh(model,
                   optimizer,
                   scheduler,
                   criterion,
                   train_loader):
    loss_history = []
    for batch in train_loader: 
        optimizer.zero_grad()
        x_train, y_train = batch # parse data
        x_train, y_train = x_train.to(device), y_train.to(device) # compute on gpu
        y_pred = model(x_train) # get predictions
        loss = criterion(y_pred, y_train) # compute loss
        loss_history.append(loss.cpu().detach().numpy()) # write loss to log
        loss.backward()
        optimizer.step()
        scheduler.step()
    return loss_history

### Запускаем

In [ ]:
model = SimpleMNIST_NN_Init_Batchnorm(n_layers=3).to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.SGD(model.parameters(),
                      lr=min_lr)

scheduler = optim.lr_scheduler.CyclicLR(optimizer,
                                        base_lr=min_lr,
                                        max_lr=max_lr,
                                        mode="triangular")  # first case

По-хорошему, в коде ниже мы на каждой эпохе сохраняем лучшую модель. 

In [ ]:
from copy import deepcopy

def train_model_cycle_sh(model, optimizer, scheduler, model_name=None, n_epochs=5):
  
    criterion = nn.CrossEntropyLoss().to(device)

    train_history = {}
    train_history['model_name'] = model_name
    train_history['loss_on_train'] = []
    train_history['loss_on_test'] = []

    best_loss = np.inf

    for epoch in tqdm(range(n_epochs)):
        model.train()
        loss_on_train = train_epoch_sh(model,
                                       optimizer,
                                       scheduler,
                                       criterion,
                                       train_loader)
        model.eval()
        val_loss, loss_on_test = validate(model,
                                   criterion,
                                   test_loader)
        train_history['loss_on_train'].extend(loss_on_train)
        train_history['loss_on_test'].extend(loss_on_test)
        if val_loss < best_loss:
            best_loss = val_loss
            best_model = deepcopy(model)
    return best_model, train_history


hooks_data_history = register_model_hooks(model)
best_model, history = train_model_cycle_sh(model,
                                           optimizer,
                                           scheduler,
                                           model_name="sgd_cycle_lr",
                                           n_epochs=15)
plot_history(history, n_epochs=15)

## Neural Network WarmUp

Также, для достаточно больших нейронных сетей практикуют следующую схему (**gradual warmup**, [изначальная статья](https://arxiv.org/pdf/1706.02677.pdf)):

Поставить изначальный learning rate значительно ниже того, с которого мы обычно начинаем обучение. За несколько эпох, например, 5, довести learning rate от этого значения до требуемого. За счет этого нейросеть лучше "адаптируется" к нашим данным. 

Также такой learning schedule позволяет адаптивным оптимизаторам лучше оценить значения learning rate для разных параметров

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L07/neural_network_warmup.png" width="1000">


kn на картинке - это размер одного батча

### Взаимодействие learning schedule и адаптивного изменения learning rate

И то, и другое меняет learning rate: learning scheduler - глобально, а адаптивные оптимизаторы - для каждого веса отдельно 

Часто их применяют вместе, особенно в случае критерия ранней остановки и WarmUp

Однако в случае циклического режима обучения так делают не всегда. Дело в том, что одна из его задач - как раз избежать использования адаптивных оптимизаторов, требующих бОльше памяти и дополнительных вычислений. 

Однако никаких препятствий к использованию того же Adam в компании вместе с циклическим режимом обучения нет. В [исходной статье](https://arxiv.org/pdf/2004.02401.pdf) так делают. 

<font size = "6"/> Ссылки:</font>

[A journey into Optimization algorithms for Deep Neural Networks](https://theaisummer.com/optimization/)

[Optimizers Explained - Adam, Momentum and Stochastic Gradient Descent](https://mlfromscratch.com/optimizers-explained/#adam)

[Батч-нормализация. In-layer normalization techniques for training very deep neural networks](https://theaisummer.com/normalization/)

[Циклический learning rate](https://towardsdatascience.com/adaptive-and-cyclical-learning-rates-using-pytorch-2bf904d18dee)

[Разные функции активации, затухающие и взрывающиеся градиенты и т.д](https://mlfromscratch.com/activation-functions-explained/#/)

[Визуализация разных оптимизаторов в ipynb, но на tensorflow](https://nbviewer.jupyter.org/github/ilguyi/optimizers.numpy/blob/master/optimizer.tf.all.opt.plot.ipynb)